# Old faithful

This exercise gives you some hands-on practice with linear regression.

The exercise comes from the Summer 2017 [Berkeley course
materials](https://github.com/data-8/data8assets) with thanks.


In [ ]:
# Run this cell, but please don't change it.

# These lines import the Numpy and Pandas modules.
import numpy as np
import pandas as pd

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# These lines load the tests.
from client.api.notebook import Notebook
ok = Notebook('old_faithful.ok')

# 1. How Faithful is Old Faithful? 

(Note: clever title comes from [here](http://web.pdx.edu/~jfreder/M212/oldfaithful.pdf).)

Old Faithful is a geyser in Yellowstone National Park in the central United States.  It's famous for erupting on a fairly regular schedule.  You can see a video below.

In [ ]:
# For the curious: this is how to display a YouTube video in a
# Jupyter notebook.  The argument to YouTubeVideo is the part
# of the URL (called a "query parameter") that identifies the
# video.  For example, the full URL for this video is:
#   https://www.youtube.com/watch?v=wE8NDuzt8eg
from IPython.display import YouTubeVideo
YouTubeVideo("wE8NDuzt8eg")

Some of Old Faithful's eruptions last longer than others.  When it has a long eruption, there's generally a longer wait until the next eruption.

If you visit Yellowstone, you might want to predict when the next eruption will happen, so you can see the rest of the park and come to see the geyser when it happens.  Today, we will use a dataset on eruption durations and waiting times to see if we can make such predictions accurately with linear regression.

The dataset has one row for each observed eruption.  It includes the following columns:

- **duration**: Eruption duration, in minutes
- **wait**: Time between this eruption and the next, also in minutes

Run the next cell to load the dataset.

In [ ]:
faithful = pd.read_csv("faithful.csv")
faithful

We would like to use linear regression to make predictions, but that
won't work well if the data aren't roughly linearly related.  To check
that, we should look at the data.


## Question 1

Make a scatter plot of the data.  It's conventional to put the column we will try to predict on the vertical axis and the other column on the horizontal axis.

In [ ]:
...

## Question 2

Look at the scatter plot. Do eruption duration and waiting time have
a rough straight line relationship?

*Write your answer here, replacing this text.*

We're going to continue with the provisional assumption that they are
linearly related, so it's reasonable to use linear regression to analyze
this data.

We'd next like to plot the data in standard units.  Recall that, if
`nums` is an array of numbers, then

    (nums - np.mean(nums)) / np.std(nums)

...is an array of those numbers in standard units.


## Question 3

Compute the mean and standard deviation of the eruption durations and
waiting times.  **Then** create a Data Frame called `faithful_standard`
containing the eruption durations and waiting times in standard units.
(The columns should be named `"duration (standard units)"` and `"wait
(standard units)"`.

In [ ]:
duration_mean = ...
duration_std = ...
wait_mean = ...
wait_std = ...
faithful_standard = pd.DataFrame()
faithful_standard["duration (standard units)"] = ...
faithful_standard["wait (standard units)"] = ...
# Show the new data frame.
faithful_standard

In [ ]:
ok.grade('q1_3');

## Question 4

Plot the data again, but this time in standard units.

In [ ]:
...

You'll notice that this plot looks exactly the same as the last one!
The data really are different, but the axes are scaled differently.
(The method `scatter` scales the axes so the data fill up the available
space.)  So it's important to read the ticks on the axes.


## Question 5

Among the following numbers, which would you guess is closest to the
correlation between eruption duration and waiting time in this dataset?

* -1
* 0
* 1

*Write your answer here, replacing this text.*


## Question 6

*Compute* the correlation `r`.  *Hint:* Use `faithful_standard`. The Correlation section in the textbook explains how to do this.

In [ ]:
r = ... 
r

In [ ]:
ok.grade('q1_6');

## 2. The regression line

Recall that the correlation is the slope of the regression line when the
data are put in standard units.

The next cell plots the regression line in standard units:

$$\text{waiting time (standard units)} = r \times \text{eruption duration (standard units)}.$$

Then, it plots the original data again, for comparison.

In [ ]:
def plot_data_and_line(dataset, x, y, point_0, point_1):
    """Makes a scatter plot of the dataset, along with a line passing through two points."""
    dataset.plot.scatter(x, y, label="data")
    xs, ys = zip(point_0, point_1)
    plt.plot(xs, ys, label="regression line", color='red')
    plt.legend(bbox_to_anchor=(1.5,.8))

plot_data_and_line(faithful_standard,
                   "duration (standard units)",
                   "wait (standard units)",
                   [-2, -2*r],
                   [2, 2*r])

## Calculating the regression line

In fact, we can calculate the regression line for the original (not
standardized) data directly from the correlation.  If you are interested
in how this works, see [the regression chapter from the Berkeley
textbook](https://www.inferentialthinking.com/chapters/15/2/Regression_Line.html)

For now, we will just re-estimate the regression using `minimize` skills.  Here is the generic least-squares line function:

In [ ]:
def ss_any_line(c_s, x_values, y_values):
    c, s = c_s
    predicted = c + x_values * s
    error = y_values - predicted
    return np.sum(error ** 2)

You will also need `scipy.optimize.minimize`

In [ ]:
from scipy.optimize import minimize

Calculate the least-squares intercept and slope for the *original* duration and wait times (not standardized):

In [ ]:
intercept, slope = ...
intercept, slope

In [ ]:
ok.grade('q2_1');


## 3. Investigating the regression line


The slope and intercept tell you exactly what the regression line looks
like.  To predict the waiting time for an eruption, multiply the
eruption's duration by `slope` and then add `intercept`.


## Question 1

Compute the predicted waiting time for an eruption that lasts 2 minutes,
and for an eruption that lasts 5 minutes.

In [ ]:
two_minute_predicted_waiting_time = ...
five_minute_predicted_waiting_time = ...
# Here is a helper function to print out your predictions
# (you don't need to modify it):
def print_prediction(duration, predicted_waiting_time):
    print("After an eruption lasting", duration,
          "minutes, we predict you'll wait", predicted_waiting_time,
          "minutes until the next eruption.")

print_prediction(2, two_minute_predicted_waiting_time)
print_prediction(5, five_minute_predicted_waiting_time)

The next cell plots the line that goes between those two points, which
is (a segment of) the regression line.

In [ ]:
plot_data_and_line(faithful, "duration", "wait",
                   [2, two_minute_predicted_waiting_time],
                   [5, five_minute_predicted_waiting_time])

## Question 2

Make predictions for the waiting time after each eruption in the
`faithful` table.  (Of course, we know exactly what the waiting times
were!  We are doing this so we can see how accurate our predictions
are.)  Put these numbers into a new variable called
`faithful_predictions`.  Its first value should be around 72.1.

*Hint:* Your answer can be just one line.  There is no need for a `for`
loop; use array arithmetic instead.

In [ ]:
faithful_predictions = ...
faithful_predictions

In [ ]:
ok.grade("q3_2");

## Question 3

How close were we?  Compute the *residual* for each eruption in the
dataset.  The residual is the actual waiting time minus the predicted
waiting time.  Make a new data frame called `faithful_residuals` that
has the original `duration` and `wait` columns, but adds the residuals
as a new column called `"residual"`

*Hint:* Again, your code will be much simpler if you don't use a `for`
loop.

In [ ]:
# Make a copy of the original table to start.
faithful_residuals = faithful.copy()
# Fill in the new column.
residuals = ...
faithful_residuals...
faithful_residuals.head()

In [ ]:
ok.grade("q3_3");

Here is a plot of the residuals you computed.  Each point corresponds to
one eruption.  It shows how much our prediction over- or under-estimated
the waiting time.

In [ ]:
faithful_residuals.plot.scatter("duration", "residual", color="r")

There isn't really a pattern in the residuals, which confirms that it
was reasonable to try linear regression.  It's true that there are two
separate clouds; the eruption durations seemed to fall into two distinct
clusters.  But that's just a pattern in the eruption durations, not a
pattern in the relationship between eruption durations and waiting
times.

## 4. How accurate are different predictions?

Earlier, you should have found that the correlation is fairly close to
1, so the line fits fairly well on the training data.  That means the
residuals are overall small (close to 0) in comparison to the waiting
times.

We can see that visually by plotting the waiting times and residuals together:

In [ ]:
faithful_residuals.plot.scatter("duration", "wait", label="actual waiting time", color="blue")
plt.scatter(faithful_residuals["duration"], faithful_residuals["residual"], label="residual", color="r")
plt.plot([2, 5], [two_minute_predicted_waiting_time, five_minute_predicted_waiting_time], label="regression line")
plt.legend(bbox_to_anchor=(1.7,.8));

However, unless you have a strong reason to believe that the linear
regression model is true, you should be wary of applying your prediction
model to data that are very different from the training data.


## Question 1

In `faithful`, no eruption lasted exactly 0, 2.5, or 60 minutes.  Using
this line, what is the predicted waiting time for an eruption that lasts
0 minutes?  2.5 minutes?  An hour?

In [ ]:
zero_minute_predicted_waiting_time = ...
two_point_five_minute_predicted_waiting_time = ...
hour_predicted_waiting_time = ...
print_prediction(0, zero_minute_predicted_waiting_time)
print_prediction(2.5, two_point_five_minute_predicted_waiting_time)
print_prediction(60, hour_predicted_waiting_time)

In [ ]:
ok.grade('q4_1');

**Question 2.** Do you believe any of these values are reliable
predictions?  If you don't believe some of them, say why.

*Write your answer here, replacing this text.*

## 5. Divide and Conquer

It appears from the scatter diagram that there are two clusters of points: one
for durations around 2 and another for durations between 3.5 and 5. A vertical
line at 3 divides the two clusters.

In [ ]:
faithful.plot.scatter("duration", "wait", label="actual waiting time", color="blue")
plt.plot([3, 3], [40, 100]);

The `standardize` function from the lecture appears below, which returns
a table of values in standard units.

In [ ]:
def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers)) / np.std(any_numbers)


def standardize(t):
    """Return table for which all columns of t converted to standard units."""
    t_su = pd.DataFrame()
    for col_name in t:
        t_su[col_name] = standard_units(t[col_name])
    return t_su

**Question 1**. Separately compute the regression coefficients r for all the
points with a duration below 3 **and then** for all the points with a duration
above 3. To do so, create a function that computes `r` from a table and pass it
two different tables of points, `below_3` and `above_3`.

In [ ]:
   ...
below_3 = ...
above_3 = ...
below_3_r = reg_coeff(below_3)
above_3_r = reg_coeff(above_3)
print("For points below 3, r is", below_3_r, "; for points above 3, r is", above_3_r)

In [ ]:
ok.grade('q5_1');

**Question 2.** Make separate lines to predict wait times.

Use `minimize` to calculate separate regression lines for durations below 3, and those above 3.

When you're done, the functions `wait_below_3` and `wait_above_3` below should
each use a different regression line to predict a wait time for a duration. The
first function should use the regression line for all points with duration
below 3. The second function should use the regression line for all points with
duration above 3.

In [ ]:
below_3_inter, below_3_slope = ...
above_3_inter, above_3_slope = ...

def wait_below_3(duration):
    return below_3_inter + below_3_slope * duration


def wait_above_3(duration):
    return above_3_inter + above_3_slope * duration

In [ ]:
ok.grade('q5_2');

The plot below shows two different regression lines, one for each cluster!

In [ ]:
faithful.plot.scatter(0, 1)
plt.plot([1, 3], [wait_below_3(1), wait_below_3(3)], color='red')
plt.plot([3, 6], [wait_above_3(3), wait_above_3(6)], color='red');

**Question 3.** Write a function `predict_wait` that takes a `duration` and
returns the predicted wait time using the appropriate regression line,
depending on whether the duration is below 3 or greater than (or equal to) 3.

In [ ]:
def predict_wait(duration):
    """ Return wait predicted by the matching regression line.

    Return wait predicted by the appropriate one of the two regression lines
    above.
    """
   ...

In [ ]:
ok.grade('q5_3');

The predicted wait times for each point appear below.

In [ ]:
faith_predicted = faithful.copy()
faith_predicted['predicted'] = faithful['duration'].apply(predict_wait)
ax = faith_predicted.plot.scatter('duration', 'wait')  # Overlay on previous.
faith_predicted.plot.scatter('duration', 'predicted', ax=ax, color='red');

**Question 4.** Do you think the predictions produced by `predict_wait` would be more or less accurate than the predictions from the regression line you created in section 2? How could you tell?

*Write your answer here, replacing this text.*




## Done

You're finished with the assignment!  Be sure to...

- **run all the tests** (the next cell has a shortcut for that),
- **Save and Checkpoint** from the "File" menu.
- Finally, **restart** the kernel for this notebook, and **run all the cells**,
  to check that the notebook still works without errors.  Use the
  "Kernel" menu, and choose "Restart and run all".  If you find any
  problems, go back and fix them, save the notebook, and restart / run
  all again, before submitting.  When you do this, you make sure that
  we, your humble markers, will be able to mark your notebook.

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]